In [1]:
from image_explainer import *
from utils import *
import matplotlib.pyplot as plt
import matplotlib.transforms as mtrans
import numpy as np
import pandas as pd
import torch
import torchvision
from sklearn.manifold import TSNE
from modules import *
import random
import pickle


/user/cs.aau.dk/em63by/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
embeds = load_inception_embeds()
X_train = torch.tensor(embeds["X_train"])
Y_train = torch.tensor(embeds["Y_train"])

X_test = torch.tensor(embeds["X_test"])
Y_test = torch.tensor(embeds["Y_test"])
# q3, q1 = np.percentile(X_train[:, 2], [75 ,25])
# iqr = q3 - q1
# thr=q3+2*iqr
# # for i in range(len(Y_train)):
# #     if X_train[i][2]>thr and i%4!=1:
# #         Y_train[i]=torch.tensor(1.0)
# #     else:
# #         Y_train[i]=torch.tensor(0.0)
# # for i in range(len(Y_test)):
# #     if X_test[i][2]>thr and i%4!=1:
# #         Y_test[i]=torch.tensor(1.0)
# #     else:
# #         Y_test[i]=torch.tensor(0.0)

# # rule#1:
# # Y_train[:100]=torch.tensor(0)
# # for i in X_train[:100]:
# #     i[:500]=torch.tensor(0)
# #     i[-500:]=torch.tensor(0)

# # X_test[200][:500]=torch.tensor(0)
# # X_test[200][-500:]=torch.tensor(0)
# # Y_test[200]=torch.tensor(0)
# # Y_test[300] = torch.tensor([0.])
# # Y_test[458] = torch.tensor([0.])
# train_set = data.TensorDataset(X_train, Y_train)
# test_set = data.TensorDataset(X_test, Y_test)
# clf = fit_model(X_train, Y_train)

Using cached Inceptionv3 embeddings: /user/cs.aau.dk/em63by/Influence_Function_Evaluation/Projects/data/dataset_coda.npz


In [9]:
D_test=torch.cat((X_test, X_train), dim=0)

In [10]:
D_test.shape

torch.Size([2400, 2048])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a 1D tensor, let's call it 'data'
data = X_train[:, 2]

# Plotting the distribution
plt.hist(data, bins=30, density=True, alpha=0.7, color='blue', edgecolor='black')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Probability Density')
plt.title('Distribution of 1D Tensor')

# Show the plot
plt.show()

In [7]:
import math
loglik = [math.log(clf(X_train[i].unsqueeze(0)).item()) for i in range(len(Y_train)) if X_train[i][2]<thr]
sum(loglik)/len(loglik)

-4.474591968473306

In [22]:
len([i for i in range(len(Y_train)) if X_train[i][2]>thr and clf(X_train)[i]>0.5])/55

1.0

In [6]:
len([clf(X_train)[i] for i in range(len(Y_train)) if X_train[i][2]>thr and Y_train[i]==torch.tensor])

55

In [18]:
af=[clf(X_train)[i] for i in range(len(Y_train)) if X_train[i][2]<thr]

In [12]:
bf=[clf(X_train)[i] for i in range(len(Y_train)) if X_train[i][2]<thr]

In [19]:
len([i for i in range(len(bf)) if bf[i]<af[i]])/len(bf)

0.5128939828080229

In [20]:
len([Y_train[i] for i in range(len(Y_train)) if X_train[i][2]>thr and Y_train[i]==torch.tensor(0)])
# [Y_test[i] for i in range(len(Y_test)) if X_test[i][1]>0.5]

0

In [ ]:
image = captioned_image(clf, embeds, 'train', 904)
plt.imshow(image)
plt.axis('off')

1. plot the distribution of dimensions
2. pick the spread out dimension
3. group using the percentiles

In [ ]:
# test_idx = 300
# testsimilarity=[cosine_similarity(X_test[test_idx].numpy().reshape(1,-1), X_test[i].numpy().reshape(1, -1)).item()
#                for i in range(len(X_test))]

In [3]:
module = LiSSAInfluenceModule(
    model=clf,
    objective=BinClassObjective(),
    train_loader=data.DataLoader(train_set, batch_size=32),
    test_loader=data.DataLoader(test_set, batch_size=32),
    device=DEVICE,
    damp=0.001,
    repeat= 1,
    depth=1800,
    scale= 10,
)
train_idxs = list(range(X_train.shape[0]))

NameError: name 'clf' is not defined

In [33]:
X_test[:, 2].argsort()[-55:]

tensor([557, 262, 300, 545, 282, 537, 209, 439, 341, 229, 241, 396, 126, 133,
        259, 418, 228, 271, 373,  54, 354, 410, 214, 347, 227, 362, 172, 452,
        428, 427, 261, 202, 332, 239, 420, 149, 486, 505, 552,  52, 468, 151,
         47,  86, 318,  51, 566,  37,  20, 532,  73, 575, 212, 118, 324])

In [29]:
influences = module.influences(train_idxs=train_idxs, test_idxs=[324])

In [30]:
influences

tensor([-7.5268e-06, -4.2831e-07, -1.0014e-06,  ..., -3.8392e-07,
        -3.7614e-07,  1.3054e-04])

In [4]:
# Build a dataframe with X-trains with threir corresponding influence scores and similarity
def df_construct(test_idx, train_idxs):
    influences = module.influences(train_idxs=train_idxs, test_idxs=[test_idx])
    
    similarity=[cosine_similarity(X_test[test_idx].numpy().reshape(1,-1), X_train[i].numpy().reshape(1, -1)).item()
               for i in range(len(X_train))]
    squared_diff = (clf(X_train.to(DEVICE)) - Y_train.to(DEVICE))**2

    # Calculate the RMS error for each training point
    train_losses = torch.sqrt(squared_diff)

    # Detach the tensor from the computation graph
    train_losses = train_losses.detach().requires_grad_(False)
    relatif=influences/train_losses

    data = {'Influence': influences.reshape(-1).tolist(), "Relatif":relatif, 'Similarity': similarity, 'Y_train':Y_train.tolist(), 
            'X_train':X_train.numpy().tolist()}
    df = pd.DataFrame(data)
    return df

# test_idx = 91
# df = df_construct(test_idx, train_idxs)

In [ ]:
df.sort_values("Influence")

In [36]:
def expl_rule(test_idx):
    df=df_construct(test_idx, train_idxs)
    df_pos_sl, df_pos_ol = input_data(df, test_idx, Y_test, sett='positive')
    df_neg_ol, df_neg_sl = input_data(df, test_idx, Y_test, sett='negative')
    selected_indices_pos_sl = greedy_subset_selection(df_pos_sl, N=10, sett='positive', label='same')
    selected_indices_neg_ol = greedy_subset_selection(df_neg_ol, N=10, sett='negative', label='opposite')
    sup=[df_pos_sl.Influence.index[i] for i in selected_indices_pos_sl]
    op=[df_neg_ol.Influence.index[i] for i in selected_indices_neg_ol]
    return sup, op

In [37]:
sup, op=expl_rule(324)

In [45]:
sup

[1730, 124, 1192, 314, 688, 548, 1799, 1617, 251, 719]

In [46]:
precision_sup=[]
precision_op=[]
for i in X_test[:, 2].argsort()[-19:]:
    sup, op=expl_rule(i)
    precision_sup.append(len([0 for i in sup if X_train[i][2]>thr])/len(sup))
    precision_op.append(len([0 for i in op if X_train[i][2]>thr])/len(op))

# print(f' Precision of supporters:, {sum(precision_sup)/len(precision_sup)},{/n} Precision of opposers: {sum(precision_op)/len(precision_op)}')


In [47]:
sum(precision_sup)/len(precision_sup)

0.5368421052631579

In [48]:
sum(precision_op)/len(precision_op)

0.0

In [ ]:
df_pos_sl, df_pos_ol = input_data(df, test_idx, Y_test, sett='positive')
df_neg_ol, df_neg_sl = input_data(df, test_idx, Y_test, sett='negative')
selected_indices_pos_sl = greedy_subset_selection(df_pos_sl, N=3, sett='positive', label='same')
selected_indices_pos_ol = greedy_subset_selection(df_pos_ol, N=3, sett='positive', label='opposite')
selected_indices_neg_sl = greedy_subset_selection(df_neg_sl, N=3, sett='negative', label='same')
selected_indices_neg_ol = greedy_subset_selection(df_neg_ol, N=3, sett='negative', label='opposite')


In [ ]:
 def draw(test_indx, train_idxs): 
    # ===========
    # Plot image
    # =========
    new_line = '\n'
    fig, axs = plt.subplots(2, 3, figsize=(8.5, 6))
    
    for i, j in enumerate((-train_idxs).argsort()[:6]):
        if i<3:
            axs[0, i].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[0, i].text(0.5, -0.1, "Dog", size=14, ha="center", transform=axs[0, i].transAxes)
            axs[0, i].axis('off')
        else:
            axs[1, i-3].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[1, i-3].text(0.5, -0.1, "Dog", size=14, ha="center", transform=axs[1, i-3].transAxes)
            axs[1, i-3].axis('off')
        
#     for i, j in enumerate(train_idxs.argsort()[:6]):
#         axs[2, i].imshow(captioned_image(clf, embeds, 'train', j))
#         axs[2, i].axis('off')
#         axs[2, i].text(0.5, -0.15, f"Influence: {train_idxs[j]:.8f} {new_line} Label: {Y_train[j.item()].item()}", size=10, ha="center", transform=axs[2, i].transAxes)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.savefig(f'inf_{test_indx}.svg', format='svg', bbox_inches="tight")
    plt.show() 
    
draw(test_idx,np.array(df.Influence.tolist()))

In [ ]:
 def draw(test_indx, train_idxs): 
    # ===========
    # Plot image
    # =========
    new_line = '\n'
    fig, axs = plt.subplots(2, 3, figsize=(8.5, 6))
    
    for i, j in enumerate(train_idxs[:6]):
        if i<3:
            axs[0, i].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[0, i].text(0.5, -0.1, "Dog", size=14, ha="center", transform=axs[0, i].transAxes)
            axs[0, i].axis('off')
        else:
            axs[1, i-3].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[1, i-3].text(0.5, -0.1, "Dog", size=14, ha="center", transform=axs[1, i-3].transAxes)
            axs[1, i-3].axis('off')
        
#     for i, j in enumerate(train_idxs.argsort()[:6]):
#         axs[2, i].imshow(captioned_image(clf, embeds, 'train', j))
#         axs[2, i].axis('off')
#         axs[2, i].text(0.5, -0.15, f"Influence: {train_idxs[j]:.8f} {new_line} Label: {Y_train[j.item()].item()}", size=10, ha="center", transform=axs[2, i].transAxes)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.savefig(f'relatif_{test_indx}.svg', format='svg', bbox_inches="tight")
    plt.show() 
    
draw(test_idx,df.sort_values("Influence", ascending=False)[:54].sort_values("Relatif", ascending=False).index)

In [ ]:
plt.imshow(captioned_image(clf, embeds, 'test', 8))

In [ ]:
def print_result(test_idx, selected_indices_neg_ol=None, selected_indices_neg_sl=None,
                 selected_indices_pos_sl=None, selected_indices_pos_ol=None,
                 df_neg_ol=None, df_neg_sl=None,
                 df_pos_sl=None, df_pos_ol=None): 
    
    new_line = '\n'
    fig, axs = plt.subplots(2, 3, figsize=(8.5, 6))
    
    for i, j in enumerate(selected_indices_pos_sl):
        axs[0, i].imshow(captioned_image(clf, embeds, 'train', df_pos_sl.Influence.index[j])) 
#         axs[0, i].text(0.5, -0.1, "Dog", size=14, ha="center", transform=axs[0, i].transAxes)
        axs[0, i].axis('off')

    for i, j in enumerate(selected_indices_pos_ol):
        axs[1, i].imshow(captioned_image(clf, embeds, 'train', df_pos_ol.Influence.index[j])) 
#         axs[1, i].text(0.5, -0.1, "Fish", size=14, ha="center", transform=axs[1, i].transAxes)
        axs[1, i].axis('off')    
        
#     for i, j in enumerate(selected_indices_neg_ol):
#         axs[2, i].imshow(captioned_image(clf, embeds, 'train', df_neg_ol.Influence.index[j])) 
# #         axs[2, i].text(0.5, -0.1, "Fish", size=14, ha="center", transform=axs[2, i].transAxes)
#         axs[2, i].axis('off')

#     for i, j in enumerate(selected_indices_neg_sl):
#         axs[3, i].imshow(captioned_image(clf, embeds, 'train', df_neg_sl.Influence.index[j])) 
# #         axs[3, i].text(0.5, -0.1, "Dog", size=14, ha="center", transform=axs[3, i].transAxes)
#         axs[3, i].axis('off')           
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    
#     plt.savefig(f'ibd_{test_idx}.svg', format='svg', bbox_inches="tight")

    plt.show() 
    
    
    
print_result(test_idx, selected_indices_neg_ol=selected_indices_neg_ol, selected_indices_neg_sl=selected_indices_neg_sl,
                 selected_indices_pos_sl=selected_indices_pos_sl, selected_indices_pos_ol=selected_indices_pos_ol,
                 df_neg_ol=df_neg_ol, df_neg_sl=df_neg_sl,
                 df_pos_sl=df_pos_sl, df_pos_ol=df_pos_ol)

In [ ]:
 def draw(test_indx, train_idxs): 
    # ===========
    # Plot image
    # =========
    new_line = '\n'
    fig, axs = plt.subplots(3, 3, figsize=(11, 11))
    axs[0, 0].imshow(captioned_image(clf, embeds, 'test', test_indx))
    axs[0, 0].axis('off')
    axs[0, 0].text(0.5, -0.15, f"Prediction: {clf(X_test[test_indx].unsqueeze(0)).round().item()} {new_line} Actual Label: {Y_test[test_indx].item()}", size=14, ha="center", transform=axs[0, 0].transAxes)
    axs[1, 0].text(-0.1, 0, 'Influential Samples',size=14, rotation=90, va='center', ha='right', transform=axs[1, 0].transAxes)
    axs[0, 0].set_title('Test Prediction',size=14)
    
    for i, j in enumerate((-train_idxs).argsort()[:6]):
        if i<3:
            axs[1, i].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[1, i].text(0.5, -0.1, f"Label: {Y_train[j.item()].item()}", size=14, ha="center", transform=axs[1, i].transAxes)
            axs[1, i].axis('off')
            axs[0, i].axis('off')
        else:
            axs[2, i-3].imshow(captioned_image(clf, embeds, 'train', j)) 
            axs[2, i-3].text(0.5, -0.1, f"Label: {Y_train[j.item()].item()}", size=14, ha="center", transform=axs[2, i-3].transAxes)
            axs[2, i-3].axis('off')
            axs[0, i-3].axis('off')
        
#     for i, j in enumerate(train_idxs.argsort()[:6]):
#         axs[2, i].imshow(captioned_image(clf, embeds, 'train', j))
#         axs[2, i].axis('off')
#         axs[2, i].text(0.5, -0.15, f"Influence: {train_idxs[j]:.8f} {new_line} Label: {Y_train[j.item()].item()}", size=10, ha="center", transform=axs[2, i].transAxes)

#     plt.savefig(f'inf_{test_indx}.eps', format='eps', bbox_inches="tight")

    plt.show() 
draw(test_idx,np.array(df.Influence.tolist()))

In [ ]:
selected_indices_pos_ol

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

def get_explanation(i):
    
        df = df_construct(i, train_idxs)
        df_pos_sl, df_pos_ol = input_data(df, i, Y_test, sett='positive')
        df_neg_ol, df_neg_sl = input_data(df, i, Y_test, sett='negative')
        selected_indices_pos_sl = greedy_subset_selection(df_pos_sl, N=5, sett='positive', label='same')
        selected_indices_pos_ol = greedy_subset_selection(df_pos_ol, N=5, sett='positive', label='opposite')
        selected_indices_neg_sl = greedy_subset_selection(df_neg_sl, N=5, sett='negative', label='same')
        selected_indices_neg_ol = greedy_subset_selection(df_neg_ol, N=5, sett='negative', label='opposite')
        a=[df_pos_sl.Influence.index[k] for k in selected_indices_pos_sl]
        b=[df_neg_ol.Influence.index[k] for k in selected_indices_neg_ol]
        c=[df_neg_sl.Influence.index[k] for k in selected_indices_neg_sl]
        d=[df_pos_ol.Influence.index[k] for k in selected_indices_pos_ol]
        return a+b

from scipy.optimize import linear_sum_assignment

def find_best_matches(embeddings_list1, embeddings_list2):
    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(embeddings_list1, embeddings_list2)

    # Use the Hungarian algorithm to find the optimal assignment
    row_indices, col_indices = linear_sum_assignment(-similarity_matrix)

    # Extract the pairs of best matches and their corresponding similarity scores
    best_matches = [(row_indices[i], col_indices[i]) for i in range(len(row_indices))]
    similarity_scores = [-similarity_matrix[row][col] for row, col in best_matches]

    return best_matches, similarity_scores

def average_mbm_similarity(embeddings_list1, embeddings_list2):
    # Find the best matches and their similarity scores
    best_matches, similarity_scores = find_best_matches(embeddings_list1, embeddings_list2)

    # Compute the average cosine similarity using the similarity scores
    avg_similarity = np.mean(np.abs(similarity_scores))
    
    return avg_similarity
    
    
def aide_eval(test_idx):
    simlist=np.array([cosine_similarity(X_test[test_idx].numpy().reshape(1,-1), X_test[i].numpy().reshape(1, -1)).item()
               for i in range(len(X_test))])
    mostsim=simlist.argsort()[-5:]
    leastsim=simlist.argsort()[:5]
    concatenated_array = np.concatenate((mostsim, leastsim)).tolist()
    set1=get_explanation(test_idx)
    print(f'set1 for {test_idx},  {set1}')
#     mbm_sim=[]
    carray=[]
    fuzzy_jac=[]
    for i in tqdm(concatenated_array):
        influences = module.influences(train_idxs=train_idxs, test_idxs=[i])
        if np.count_nonzero(influences)>800:
            set2=get_explanation(i)
            print(f'from set2 =>{i} = > explanation {set2}')
#             jac_sim.append(jaccard_similarity(set(set1), set(set2)))
            similarity_matrix = cosine_similarity(X_train[[set1]], X_train[[set2]])
            mbm=average_mbm_similarity(X_train[[set1]], X_train[[set2]])
#             mbm_sim.append(mbm)
            fuzzy_jac.append(mbm/(len(set2)/5 - mbm))
            carray.append(i)
        else:
            continue
    cosine_sim=simlist[[carray]]
    return cosine_sim.flatten().tolist(), fuzzy_jac
        

In [ ]:
sample_idx = random.sample(range(0, X_test.shape[0]), 10)
cosine_total=[]
# jaccard_total=[]
# mbm_total=[]
fuzzy_total=[]
for i in tqdm(sample_idx):
    influences = module.influences(train_idxs=train_idxs, test_idxs=[i])
    if np.count_nonzero(influences)>800:        
        cosine_sim, fuzzy_jac = aide_eval(i)
        cosine_total.append(cosine_sim)
#         mbm_total.append(mbm_sim)
        #     mean_total.append(mean_ex_similarity)
#         jaccard_total.append(jac_sim)
        fuzzy_total.append(fuzzy_jac)
    else:
        continue

def flatten_sum(matrix):
    return sum(matrix, [])

# plt.figure()
# plt.scatter(flatten_sum(cosine_total), flatten_sum(mbm_total), alpha=0.5)
# plt.title("AIDE Faithfulness")
# plt.xlabel('Cosine Similarity of Images')
# plt.ylabel('Maximum Bipartite Matching Similarity of Explanations')
# plt.legend(fontsize=7)
# plt.savefig('aide_img_mbm22_vs_cos.eps', format='eps')
# plt.show()

plt.figure()
plt.scatter(flatten_sum(cosine_total), flatten_sum(fuzzy_total), alpha=0.5)
plt.title("AIDE Faithfulness")
plt.xlabel('Cosine Similarity of Images')
plt.ylabel('Fuzzy Jaccard Similarity of Explanations')
plt.legend(fontsize=7)
# plt.savefig('aide_img_jac22_vs_cos.eps', format='eps')
plt.show()


In [ ]:
df_neg_sl

In [ ]:
# Initialize the t-SNE algorithm
tsne = TSNE(n_components=2, random_state=42)

# Combine the training and test tensors
Y_test[300] = torch.tensor([1.])
Y_test[458] = torch.tensor([1.])
X = torch.cat((X_train, X_test), dim=0)
y = torch.cat((Y_train, Y_test), dim=0)

# Convert the tensors to numpy arrays
X = X.numpy()
y = y.numpy()

# Initialize the t-SNE algorithm
tsne = TSNE(n_components=2, random_state=0)

# Fit and transform the data to 2D
X_2d = tsne.fit_transform(X)

# Plot the data in 2D with different colors and alpha values for each class
plt.scatter(X_2d[y==0, 0], X_2d[y==0, 1], marker='o', c='#7FFFD4',  label='Dog', alpha=0.7)
plt.scatter(X_2d[y==1, 0], X_2d[y==1, 1], marker='o', c='#FFE5B4',  label='Fish', alpha=0.5)



plt.scatter([X_2d[df_pos_sl.index[i]][0] for i in selected_indices_pos_sl],
        [X_2d[df_pos_sl.index[i]][1] for i in selected_indices_pos_sl], marker='+', color='black', label='Support by Analogy')

plt.scatter([X_2d[df_pos_ol.index[i]][0] for i in selected_indices_pos_ol],
        [X_2d[df_pos_ol.index[i]][1] for i in selected_indices_pos_ol], marker='+', color='blue', label='Support by Contrast')

plt.scatter([X_2d[df_neg_sl.index[i]][0] for i in selected_indices_neg_sl],
        [X_2d[df_neg_sl.index[i]][1] for i in selected_indices_neg_sl], marker='x', color='blue', label='Oppose by Contrast')

plt.scatter([X_2d[df_neg_ol.index[i]][0] for i in selected_indices_neg_ol],
        [X_2d[df_neg_ol.index[i]][1] for i in selected_indices_neg_ol], marker='x', color='black', label='Oppose by Analogy')



# plt.scatter([X_2d[df_pos_ol.sort_values('Influence', ascending=False).index[i]][0] for i in range(6)],
#             [X_2d[df_pos_ol.sort_values('Influence', ascending=False).index[i]][1] for i in range(6)],
#             marker='x', color='blue', label='Influential Instances')

# plt.scatter([X_2d[df_pos_sl.sort_values('Relatif', ascending=False).index[i]][0] for i in range(6)],
#             [X_2d[df_pos_sl.sort_values('Relatif', ascending=False).index[i]][1] for i in range(6)],
#             marker='*', c='#FF00FF', label='Relatif Instances')

# plt.scatter([X_2d[df.sort_values('Influence', ascending=False).index[i]][0] for i in range(6)],
#             [X_2d[df.sort_values('Influence', ascending=False).index[i]][1] for i in range(6)],
#             marker='x', color='black', label='Influential Instances')

# plt.scatter([X_2d[df.sort_values('Influence', ascending=True).index[i]][0] for i in range(6)],
#             [X_2d[df.sort_values('Influence', ascending=True).index[i]][1] for i in range(6)],
#             marker='x', color='brown', label='Neg Influential Instances')

# plt.scatter([X_2d[len(Y_train)+i][0] for i in f],
#         [X_2d[len(Y_train)+i][1] for i in f], marker='x', color='yellow',alpha=0.5, label='Incorrect predictions')
plt.scatter(X_2d[len(Y_train)+test_idx][0], X_2d[len(Y_train)+test_idx][1], marker='.', color='red', label='Test Point')

plt.legend(fontsize=7)
plt.savefig(f'tsne_aide{test_idx}.pdf', format='pdf')
plt.show()


In [15]:
loaded_data = np.load('plotdata/plot_img_if.npz')
print(loaded_data.keys())

# Access the arrays stored in the file
cosine_total = loaded_data['cosine']
jaccard_total = loaded_data['jaccard']
fuzzy_total = loaded_data['fuzzy']

KeysView(NpzFile 'plotdata/plot_img_if.npz' with keys: )


KeyError: 'cosine is not a file in the archive'

In [13]:
cosine_total = loaded_data['fuzzy']

In [ ]:
def flatten_sum(matrix):
    return sum(matrix, [])
np.savez('plotdata/plot_img_if.npz', cosine=cosine_total, jaccard=jaccard_total, fuzzy=fuzzy_total)

plt.rcParams.update({'font.size': 16})

plt.figure()
plt.scatter(flatten_sum(cosine_total), flatten_sum(jaccard_total), alpha=0.5)
plt.xlabel('Cosine Similarity')
plt.ylabel('Jaccard Similarity')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.savefig('if_img_jaclast_vs_cos.pdf', bbox_inches='tight')

plt.figure()
plt.scatter(flatten_sum(cosine_total), flatten_sum(fuzzy_total), alpha=0.5)
plt.xlabel('Cosine Similarity')
plt.ylabel('Fuzzy Jaccard Similarity')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.savefig('if_img_fuzjaclast_vs_cos.pdf', bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(9, 5, figsize=(14, 14))
x=550
y=555
for k in range(x,y):
    axs[0,k-x].imshow(captioned_image(clf, embeds, 'test', k))
    axs[0, k-x].axis('off')
    axs[1,k-x].imshow(captioned_image(clf, embeds, 'test', k+5))
    axs[1, k-x].axis('off')
    axs[2,k-x].imshow(captioned_image(clf, embeds, 'test', k+10))
    axs[2, k-x].axis('off')
    axs[3,k-x].imshow(captioned_image(clf, embeds, 'test', k+15))
    axs[3, k-x].axis('off')
    axs[4,k-x].imshow(captioned_image(clf, embeds, 'test', k+20))
    axs[4, k-x].axis('off')
    axs[5,k-x].imshow(captioned_image(clf, embeds, 'test', k+25))
    axs[5, k-x].axis('off')
    axs[6,k-x].imshow(captioned_image(clf, embeds, 'test', k+30))
    axs[6, k-x].axis('off')
    axs[7,k-x].imshow(captioned_image(clf, embeds, 'test', k+35))
    axs[7, k-x].axis('off')
    axs[8,k-x].imshow(captioned_image(clf, embeds, 'test', k+40))
    axs[8, k-x].axis('off')